In [8]:
import os
import cv2
import numpy as np
from imutils import face_utils
import dlib

In [9]:
input_folder = "D:\\2. College project\\sem 5 drowsiness detection while driving\\driver_drowsiness_system_CNN\\archive\\train"
processed_folder = "D:\\2. College project\\sem 5 drowsiness detection while driving\\image-processing\\new_test_processing"
# face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor('data/shape_predictor_68_face_landmarks.dat')

# Image Processing

In [10]:
def remove_noise_gaussian(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

def sharpen_image(image, intensity=1.0):
    """
    Sharpen the image using kernel.
    Intensity: Low (0.5) to Medium (1.0). Higher intensity may damage quality.
    """
    kernel = np.array([
        [0, -1, 0],
        [-1, 4 + intensity, -1],
        [0, -1, 0]
    ])
    sharpened = cv2.filter2D(image, -1, kernel)
    return cv2.addWeighted(image, 1.0, sharpened, 0.5, 0)

In [11]:
def align_face(image, rect, landmarks):
    left_eye = landmarks[36:42].mean(axis=0)
    right_eye = landmarks[42:48].mean(axis=0)
    dx = right_eye[0] - left_eye[0]
    dy = right_eye[1] - left_eye[1]
    angle = np.degrees(np.arctan2(dy, dx))
    center = ((rect.left() + rect.right()) // 2, (rect.top() + rect.bottom()) // 2)
    rot_matrix = cv2.getRotationMatrix2D(center, angle, scale=1.0)
    aligned_image = cv2.warpAffine(image, rot_matrix, (image.shape[1], image.shape[0]))
    return aligned_image

In [12]:
def preprocess_yawn_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) 
    if img is None:
        print(f"Failed to read image: {image_path}")
        return None

    img_denoised = remove_noise_gaussian(img)

    img_sharpened = sharpen_image(img_denoised, intensity=0.8)

    detector = dlib.get_frontal_face_detector()
    faces = detector(img_sharpened, 1)
    if len(faces) == 0:
        print(f"No face detected in image: {image_path}")
        return None

    for face in faces:
        landmarks = predictor(img_sharpened, face)
        landmarks = face_utils.shape_to_np(landmarks)

        aligned_img = align_face(img_sharpened, face, landmarks)

        aligned_landmarks = predictor(aligned_img, face)
        aligned_landmarks = face_utils.shape_to_np(aligned_landmarks)

        mouth_points = aligned_landmarks[48:68]
        (x_min, y_min) = mouth_points.min(axis=0)
        (x_max, y_max) = mouth_points.max(axis=0)

        # add margin biar croppingnya ga kecil bgt
        margin = 15
        x_min = max(0, x_min - margin)
        y_min = max(0, y_min - margin)
        x_max = min(aligned_img.shape[1], x_max + margin)
        y_max = min(aligned_img.shape[0], y_max + margin)

        mouth = aligned_img[y_min:y_max, x_min:x_max]

        mouth_resized = cv2.resize(mouth, (128, 128), interpolation=cv2.INTER_CUBIC)
        mouth_normalized = mouth_resized / 255.0

        return mouth_normalized

In [13]:
def preprocess_eye_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) 
    if img is None:
        print(f"Failed to read image: {image_path}")
        return None

    img_denoised = remove_noise_gaussian(img)

    img_sharpened = sharpen_image(img_denoised, intensity=0.5)

    eye_resized = cv2.resize(img_sharpened, (128, 128), interpolation=cv2.INTER_CUBIC)
    eye_normalized = eye_resized / 255.0

    return eye_normalized

In [14]:
def process_and_save_images(input_folder, processed_folder):
    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)

    for category in os.listdir(input_folder):
        category_path = os.path.join(input_folder, category)
        output_category_folder = os.path.join(processed_folder, category)
        if not os.path.exists(output_category_folder):
            os.makedirs(output_category_folder)

        for filename in os.listdir(category_path):
            image_path = os.path.join(category_path, filename)

            if category in ['yawn', 'no_yawn']:
                processed_image = preprocess_yawn_image(image_path)
            elif category in ['Closed', 'Open']:
                processed_image = preprocess_eye_image(image_path)
            else:
                print(f"Skipped unknown category: {category}")
                continue

            if processed_image is not None:
                processed_image = (processed_image * 255).astype(np.uint8)
                output_path = os.path.join(output_category_folder, filename)
                cv2.imwrite(output_path, processed_image)
                print(f"Processed and saved: {output_path}")

process_and_save_images(input_folder, processed_folder)

Processed and saved: D:\2. College project\sem 5 drowsiness detection while driving\image-processing\new_test_processing\Closed\_0.jpg
Processed and saved: D:\2. College project\sem 5 drowsiness detection while driving\image-processing\new_test_processing\Closed\_1.jpg
Processed and saved: D:\2. College project\sem 5 drowsiness detection while driving\image-processing\new_test_processing\Closed\_10.jpg
Processed and saved: D:\2. College project\sem 5 drowsiness detection while driving\image-processing\new_test_processing\Closed\_100.jpg
Processed and saved: D:\2. College project\sem 5 drowsiness detection while driving\image-processing\new_test_processing\Closed\_101.jpg
Processed and saved: D:\2. College project\sem 5 drowsiness detection while driving\image-processing\new_test_processing\Closed\_102.jpg
Processed and saved: D:\2. College project\sem 5 drowsiness detection while driving\image-processing\new_test_processing\Closed\_103.jpg
Processed and saved: D:\2. College project\sem